# Mutual information for features with missing values

*From my stackexchange question and answer [link](https://stats.stackexchange.com/questions/661624/encoding-high-proportion-of-missing-values-for-continuous-variable-results-in-mu/661625)

Calculating mutual information using `sklearn.feature_selection.mutual_info_classif` for a feature with high proportion of missing values can give misleading results. Especially when there are many entries with the same value of a feature (like e.g. many missing values encoded as single number).

Let's consider extreme example:
- `X`: 10 000 numbers drawn from $Exponential(1)$ distribution and 80 000 missing values
- `y`: binary variable with only 100 "ones" (only about 0.15%) which is completely independent of `X`

Since `mutual_info_classif` doesn't accept missing values have to be encoded as a value outside range of non-missing part (e.g. `-999`).

In [22]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
from scipy.special import xlogy

In [ ]:
rng = np.random.default_rng(seed=42)
X = rng.exponential(1, 10_000)
# Simulate encoding nulls
X = np.append(X, -999 * np.ones(80_000))

# Completely independent target
y = np.zeros_like(X)
y_1 = rng.choice(list(range(len(X))), 100, replace=False)
y[y_1] = 1

print(f"Entropy of y from mutual_info_classif:                    H(y) = {mutual_info_classif(y[:,None], y, discrete_features=True)[0]}")
print(f"Entropy of y calculated:                                  H(y) = {xlogy(y.mean(), 1/y.mean())+xlogy(1-y.mean(), 1/(1-y.mean()))}")
print(f"Mutual information of X and y from mutual_info_classif: I(X,y) = {mutual_info_classif(X[:,None], y)[0]}")

Entropy of y from mutual_info_classif:                    H(y) = 0.008668710002102975
Entropy of y calculated:                                  H(y) = 0.008668710002103328
Mutual information of X and y from mutual_info_classif: I(X,y) = 0.016368695523635624


This result does not fulfill the property of mutual information
$$
I(X; Y) \leq \min(H(X), H(Y))
$$

This is caused by how [KSG estimator](https://arxiv.org/abs/cond-mat/0305641) works (which is what is used in `mutual_info_classif`). It is based on idea of nearest-neighbours counting, which might give incorrect results in case when there are duplicates in the continuous feature. It is partially solved by addition of small amount of noise to the feature (which is why there is `random_state` parameter in `mutual_info_classif` which controls the seed for this noise) but in such extreme cases it fails.

This problem can be solved by introduction of helper binary variable $X_{NA}$ that indicates whether the feature is null or not for a given observation. Feature can also be interpreted as *mixed random variable* where missingness is discrete part and the rest is continuous part. Then using chain-rule of mutual information we get this formula:
$$
\begin{split}
I(X; Y) &= I(X_{NA}; Y) + I(X;Y\mid X_{NA})\\
&= I(X_{NA}; Y) + P(X_{NA}=0) I(X\mid X_{NA}=0;Y\mid X_{NA}=0)\\
&= I(X_{NA}; Y) + P(X_{NA}=0) I_c(X;Y)
\end{split}
$$
where
- $X_{NA}=1$ when $X$ is missing and $X_{NA}=0$ when $X$ is not missing
- $X\mid X_{NA}=0$ and $Y\mid X_{NA}=0$ is a subset of the data where $X$ is *non-missing*
- $I_c(X;Y)$ is mutual information estimation for part of the data that has *non-mising* $X$

So first we calculate mutual information of missingness and $Y$ and then we add to it mutual information of *non-missing part* scaled by probability of observing *non-missing part*. This form is quite useful since it allows using existing scikit-learn functions.

In [ ]:
def mutual_info_classif_mixed(x, y, special_code=None, **kwargs):
    if not isinstance(x, pd.Series):
        x = pd.Series(x)

    if not isinstance(y, pd.Series):
        y = pd.Series(y)

    # Binary variable indicating "missingness" (either for NaNs or encoded with special_code)
    if special_code is None:
        x_d = (x.isna()).astype(int)
    else:
        x_d = (x == special_code).astype(int)

    # Non-missing part of variables
    x_c = x[x_d == 0]
    y_c = y[x_d == 0]

    # Discrete part of MI
    mi_d = mutual_info_classif(
        x_d.values[:, None], y, discrete_features=True, **kwargs
    )[0]

    # Continuous part of MI
    mi_c = mutual_info_classif(
        x_c.values[:, None], y_c, discrete_features=False, **kwargs
    )[0]

    return mi_d + (1 - x_d.mean()) * mi_c

In [24]:
rng = np.random.default_rng(seed=42)
n = 1000
entropy_list = []
mi_list = []
mi_mixed_list = []
for i in range(n):
    print(i, flush=True, end="\r")
    X = rng.exponential(1, 10_000)
    X = np.append(X, -999 * np.ones(80_000))

    y = np.zeros_like(X)
    y_1 = rng.choice(list(range(len(X))), 100, replace=False)
    y[y_1] = 1

    entropy_list.append(mutual_info_classif(y[:,None],y, discrete_features=True)[0])
    mi_list.append(mutual_info_classif(X[:,None],y)[0])
    mi_mixed_list.append(mutual_info_classif_mixed(X, y, special_code=-999))

entropy_array = np.array(entropy_list)
mi_array = np.array(mi_list)
mi_mixed_array = np.array(mi_mixed_list)

print(f"Entropy of y from mutual_info_classif:                    H(y) = {entropy_array.mean()} +/- {entropy_array.std()}")
print(f"Entropy of y calculated:                                  H(y) = {xlogy(y.mean(), 1/y.mean())+xlogy(1-y.mean(), 1/(1-y.mean()))}")
print(f"Mutual information of X and y from mutual_info_classif: I(X,y) = {mi_array.mean()} +/- {mi_array.std()}")
print(f"Corrected mutual_info_classif:                          I(X,y) = {mi_mixed_array.mean()} +/- {mi_mixed_array.std()}")

Entropy of y from mutual_info_classif:                    H(y) = 0.008668710002102977 +/- 1.734723475976807e-18
Entropy of y calculated:                                  H(y) = 0.008668710002103328
Mutual information of X and y from mutual_info_classif: I(X,y) = 0.01610285624719649 +/- 0.00040891585424112536
Corrected mutual_info_classif:                          I(X,y) = 1.680313074239922e-05 +/- 2.0923604220299665e-05
